# searching + downloading sentinel data using sentinelsat

In [ ]:
import geopandas as gpd
from sentinelsat import SentinelAPI, make_path_filter
from IPython import display
import os

## prepare a search area

In [ ]:
rgi_dir = '/media/bob/seabox/RGI/v6.0/' # change this to be wherever you've stored the RGI
rgi_alaska = gpd.read_file(os.path.join(rgi_dir, '01_rgi60_Alaska/01_rgi60_Alaska.shp'))

# gets all of the region 01-01 (North Alaska) glaciers
brooks_range = rgi_alaska[rgi_alaska['O2Region'] == '1']['geometry'].unary_union

# gets a well-known text (wkt) representation of the smallest rectangle that covers the glaciers
search_area_wkt = brooks_range.minimum_rotated_rectangle.wkt

# displays the search area wkt
print(search_area_wkt)

## search the archive for images

### connect to the api

In [ ]:
api = SentinelAPI(None, None) # replace None, None with <username>, <password> if you don't have a .netrc file

### search for images

In [ ]:
products = api.query(search_area_wkt,
                     date=('20210801', '20210930'), # all images from 1 August 2021 to 30 Sept 2021
                     platformname='Sentinel-2',
                     producttype='S2MSI2A', # surface reflectance product (L2A)
                     cloudcoverpercentage=(0, 10), # limit to 10% cloud cover
                     snowicepercentage=(0, 40)) # limit to 40% snow/ice cover

In [ ]:
result = next(iter(products)) # get the next item from the ordereddict

products[result] # show the metadata for the first item

### view browse images

In [ ]:
qlook = api.download_quicklook(result) # download the quicklook image for the first result

display.Image(qlook['path']) # display the image in the browser

### download an individual image

In [ ]:
api.download(first_result, # downloads the first result
             nodefilter=make_path_filter("*_B*.jp2")) # only download the image bands (optional)

### download all images from a list of products

In [ ]:
api.download_all(products,
                 n_concurrent_dl=5, # allow up to 5 concurrent downloads
                 nodefilter=make_path_filter("*_B*.jp2")) # only down the image bands (optional)